# DNN Model Notebook

```md
@authors: miguelrocha and Grupo 03
```

In [1]:
# Notebook Imports
import numpy as np

from models.dnn_model import NeuralNetwork, hyperparameter_optimization
from helpers.dataset import Dataset
from helpers.enums import ModelRunMode
from helpers.layers import DenseLayer
from helpers.activation import ReLUActivation, SigmoidActivation
from helpers.losses import BinaryCrossEntropy
from helpers.optimizer import Optimizer
from helpers.metrics import accuracy, confusion_matrix, balanced_accuracy, precision_recall_f1
from helpers.enums import ModelType
from helpers.model import save_dnn_model

In [2]:
# Model run mode
# Options: 
#   ModelRunMode.TRAIN.value            (Train the model)
#   ModelRunMode.CLASSIFY.value         (Classify data)
mode = ModelRunMode.TRAIN.value
# Prefix for saving the model files
model_prefix = "dnn_model_1"

In [3]:
# Parameters cell
if mode == ModelRunMode.TRAIN.value:
    # TRAIN mode: Set parameters for training
    input_csv = "../tarefa_1/test_input_dataset/merged_inputs.csv"                      # CSV file with training inputs (ID, Text)
    output_csv = "../tarefa_1/test_output_dataset/merged_outputs.csv"                   # CSV file with training outputs (ID, Label)
    test_size = 0.3                                                                     # Proportion of the dataset to use as test data
    validation_size = 0.2                                                               # Proportion of the dataset reserved as validation data (used during hyperparameter tuning)
    verbose = True                                                                      # Verbosity level enabler
    random_state=42                                                                     # Seed for reproducible dataset splitting
    max_vocab_size=1000                                                                 # Maximum vocabulary size (None implies no limit)
    min_freq=1                                                                          # Minimum frequency required for a word to be included in the vocabulary
elif mode == ModelRunMode.CLASSIFY.value:
    # CLASSIFY mode: Set parameters for classification
    input_csv = "../tarefa_2/classify_input_datasets/submission3_inputs.csv"               # CSV file with texts for prediction (ID, Text)
    output_csv = "../tarefa_2/classify_output_datasets/submission3_outputs_dnn_model.csv"  # CSV file to store prediction result
else:
    print("The selected option is not valid. Options: \"train\" or \"classify\"!")
    SystemExit()

In [4]:
if mode == ModelRunMode.TRAIN.value:
    # Load datasets using TF-IDF vectorization
    X_train, y_train, X_test, y_test, vocab, idf = Dataset.prepare_train_test_tfidf(input_csv=input_csv, output_csv=output_csv, test_size=test_size, random_state=random_state, max_vocab_size=max_vocab_size, min_freq=min_freq)

    # Create Dataset objects for training and test data
    train_ds_full = Dataset(X=X_train, Y=y_train)
    test_ds = Dataset(X=X_test, Y=y_test)

    # Display dimensions of the training and test datasets
    print(f"Train set has {train_ds_full.nrows()} rows and {train_ds_full.ncols()} columns")
    print(f"Test set has {test_ds.nrows()} rows and {test_ds.ncols()} columns\n")

    # Split the full training dataset into training and validation subsets (70% training, 30% validation)
    n_train = train_ds_full.X.shape[0]
    indices = np.arange(n_train)
    np.random.shuffle(indices)
    split_idx = int((1 - validation_size) * n_train)
    train_idx = indices[:split_idx]
    val_idx = indices[split_idx:]
    train_ds = Dataset(X=train_ds_full.X[train_idx], Y=train_ds_full.Y[train_idx])
    val_ds = Dataset(X=train_ds_full.X[val_idx], Y=train_ds_full.Y[val_idx])

    # More varied epochs to handle both simpler and more complex tasks
    epochs_list = [100, 150]

    # A wider range of batch sizes
    batch_size_list = [32, 64]

    # Learning rate from very small to moderate
    learning_rate_list = [0.001, 0.01]

    # Momentum slightly varied around typical defaults
    momentum_list = [0.8, 0.9]

    # Different hidden layer topologies
    hidden_layers_list = [
        [64],
        [64, 32],
        [128, 64]
    ]

    # Add mild to moderate dropout levels
    dropout_list = [0.0, 0.1]

    # Perform hyperparameter tuning using the training and validation sets
    print("Starting hyperparameter tuning...")
    best_params = hyperparameter_optimization(train_ds, val_ds, epochs_list, batch_size_list, learning_rate_list, momentum_list, hidden_layers_list, dropout_list, n_iter=96)
    print("\nBest hyperparameters:", best_params)

    # Retrain DNN model on the full training data using the best hyperparameters
    final_model = NeuralNetwork(epochs=best_params['epochs'], batch_size=best_params['batch_size'], optimizer=Optimizer(learning_rate=best_params['learning_rate'], momentum=best_params['momentum']), verbose=verbose, loss=BinaryCrossEntropy,  metric=accuracy)

    # Build model Layers
    n_features = train_ds.X.shape[1]
    for i, units in enumerate(best_params['n_hidden']):
        if i == 0:
            final_model.add(DenseLayer(units, (n_features,),dropout_rate=best_params['dropout_rate']))
        else:
            final_model.add(DenseLayer(units,dropout_rate=best_params['dropout_rate']))
        final_model.add(ReLUActivation())
        
    final_model.add(DenseLayer(1))
    final_model.add(SigmoidActivation())

    # Fit model
    final_model.fit(train_ds)

    # Get predictions on the test set
    preds = final_model.predict(test_ds)

    # Convert probabilities to binary class labels using a threshold of 0.5
    preds_binary = (preds > 0.5).astype(int)

    # Evaluate the final model on the test set
    test_acc = final_model.score(test_ds, preds)
    print(f"\nTest accuracy with best hyperparameters: {test_acc:.4f}")

    # Evaluate the final model using binary predictions for the confusion matrix and other metrics
    TP, FP, TN, FN = confusion_matrix(y_test, preds_binary)
    prec, rec, f1 = precision_recall_f1(y_test, preds_binary)
    bal_acc = balanced_accuracy(y_test, preds_binary)

    print("Confusion Matrix: TP={}, FP={}, TN={}, FN={}".format(TP, FP, TN, FN))
    print("Precision = {:.4f}, Recall = {:.4f}, F1 = {:.4f}".format(prec, rec, f1))
    print("Balanced Accuracy = {:.4f}".format(bal_acc))

    # Save the model, plus vocab & idf, so classification can replicate
    save_dnn_model(dnn=final_model, vocab=vocab, idf=idf, model_prefix=model_prefix)
    print(f"DNN Model saved with prefix {model_prefix}")

if mode == ModelRunMode.CLASSIFY.value:
    # Classify new texts using the saved model
    Dataset.classify_texts(input_csv=input_csv, output_csv=output_csv, neural_net_class=NeuralNetwork, model_type=ModelType.DNN ,model_prefix=model_prefix)

Train set has 1319 rows and 1000 columns
Test set has 565 rows and 1000 columns

Starting hyperparameter tuning...


Hyperparameter Search:   1%|          | 1/96 [00:03<05:41,  3.60s/it]

Parameters: {'epochs': 100, 'batch_size': 64, 'learning_rate': 0.01, 'momentum': 0.9, 'n_hidden': [64], 'dropout_rate': 0.0}  |  Accuracy: 0.9659


Hyperparameter Search:   2%|▏         | 2/96 [00:15<13:42,  8.75s/it]

Parameters: {'epochs': 150, 'batch_size': 64, 'learning_rate': 0.01, 'momentum': 0.8, 'n_hidden': [128, 64], 'dropout_rate': 0.1}  |  Accuracy: 0.9394


...

Hyperparameter Search:  99%|█████████▉| 95/96 [10:44<00:06,  6.38s/it]

Parameters: {'epochs': 100, 'batch_size': 32, 'learning_rate': 0.01, 'momentum': 0.9, 'n_hidden': [64], 'dropout_rate': 0.1}  |  Accuracy: 0.9545


Hyperparameter Search: 100%|██████████| 96/96 [10:47<00:00,  6.75s/it]

Parameters: {'epochs': 100, 'batch_size': 64, 'learning_rate': 0.001, 'momentum': 0.9, 'n_hidden': [64, 32], 'dropout_rate': 0.1}  |  Accuracy: 0.8750

Best Hyperparameters Found: {'epochs': 150, 'batch_size': 32, 'learning_rate': 0.01, 'momentum': 0.8, 'n_hidden': [64], 'dropout_rate': 0.1}
Best Accuracy: 0.9697

Best hyperparameters: {'epochs': 150, 'batch_size': 32, 'learning_rate': 0.01, 'momentum': 0.8, 'n_hidden': [64], 'dropout_rate': 0.1}
Epoch 1/150 - loss: 0.6866 - accuracy: 0.5625
Epoch 2/150 - loss: 0.6563 - accuracy: 0.6543
Epoch 3/150 - loss: 0.6333 - accuracy: 0.6826
Epoch 4/150 - loss: 0.6135 - accuracy: 0.7031


Epoch 5/150 - loss: 0.5868 - accuracy: 0.7432
Epoch 6/150 - loss: 0.5497 - accuracy: 0.7812
Epoch 7/150 - loss: 0.5166 - accuracy: 0.8086
Epoch 8/150 - loss: 0.4745 - accuracy: 0.8340
Epoch 9/150 - loss: 0.4389 - accuracy: 0.8447
Epoch 10/150 - loss: 0.3980 - accuracy: 0.8613
Epoch 11/150 - loss: 0.3694 - accuracy: 0.8838
Epoch 12/150 - loss: 0.3408 - accuracy: 0.8867
Epoch 13/150 - loss: 0.3005 - accuracy: 0.9043
Epoch 14/150 - loss: 0.2816 - accuracy: 0.9229
Epoch 15/150 - loss: 0.2601 - accuracy: 0.9326
Epoch 16/150 - loss: 0.2394 - accuracy: 0.9336
Epoch 17/150 - loss: 0.2254 - accuracy: 0.9355
Epoch 18/150 - loss: 0.1940 - accuracy: 0.9502
Epoch 19/150 - loss: 0.1771 - accuracy: 0.9570
Epoch 20/150 - loss: 0.1691 - accuracy: 0.9502
Epoch 21/150 - loss: 0.1564 - accuracy: 0.9600
Epoch 22/150 - loss: 0.1437 - accuracy: 0.9658
Epoch 23/150 - loss: 0.1398 - accuracy: 0.9590
Epoch 24/150 - loss: 0.1219 - accuracy: 0.9756
Epoch 25/150 - loss: 0.1129 - accuracy: 0.9775
Epoch 26/150 - los